In [0]:
#rahul pandit
from google.colab import drive
drive.mount('/content/drive')
import os
os.getcwd()
os.chdir('/content/drive/My Drive/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np 
import pandas as pd
import random as rn
import cv2 as cv 
import os
import sys
from pathlib import Path
import ast 
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

<a id="4"></a> <br>
# Visualising the Masks
Let's take a look at some examples of each class, and of some of the images containing multiple classes.

In [0]:
""" use a consistent color palette per label throughout the notebook """
import colorlover as cl

# see: https://plot.ly/ipython-notebooks/color-scales/
colors = cl.scales['4']['qual']['Set3']
labels = np.array(range(1,5))
palette = dict(zip(labels, np.array(cl.to_numeric(colors))))

In [0]:
def rle_to_mask(lre, shape=(1600,256)):
    runs = np.asarray([int(run) for run in lre.split(' ')])
    runs[1::2] += runs[0::2]
    runs -= 1
    run_starts, run_ends = runs[0::2], runs[1::2]
    h, w = shape
    mask = np.zeros(h*w, dtype=np.uint8)
    for start, end in zip(run_starts, run_ends):
        mask[start:end] = 1
    return mask.reshape(shape)

In [0]:
def build_mask(encodings, labels):
    mask = np.zeros((256,1600,4), dtype=np.uint8)
    for rle, label in zip(encodings, labels):
        index = label - 1
        mask[:,:,index] = rle_to_mask(rle).T    
    return mask

In [0]:
def mask_to_contours1(image, mask_layer, color):
    contours, hierarchy = cv.findContours(mask_layer, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    return contours

In [0]:
def get_contour_train(file_name,mask):
    contours=[]
    image = cv.imread("train/{}".format(file_name))
    for index in range(mask.shape[-1]):
        data_dict={}
        data_dict['Image_id']=[]
        data_dict['Class_id']=[]
        data_dict['contour']=[]
      
        label = index + 1
        contour=mask_to_contours1(image, mask[:,:,index], color=palette[label])
        if len(contour)>0:

            data_dict['Image_id'].append(file_name)
            data_dict['Class_id'].append(index)
            data_dict['contour'].append(contour)
            contours.append(data_dict)
    return contours


In [0]:
def get_contour_val(file_name,mask):
    contours=[]
    image = cv.imread("val/{}".format(file_name))
    for index in range(mask.shape[-1]):
        data_dict={}
        data_dict['Image_id']=[]
        data_dict['Class_id']=[]
        data_dict['contour']=[]
      
        label = index + 1
        contour=mask_to_contours1(image, mask[:,:,index], color=palette[label])
        if len(contour)>0:

            data_dict['Image_id'].append(file_name)
            data_dict['Class_id'].append(index)
            data_dict['contour'].append(contour)
            contours.append(data_dict)
    return contours


#Algo for Detectron-2


In [0]:
df=pd.read_csv('steel-defect/train.csv')
df['EncodedPixels']=df['EncodedPixels'].apply(lambda x:ast.literal_eval(x))
df['ClassId']=df['ClassId'].apply(lambda x:ast.literal_eval(x))
df=df.iloc[1:5,:]
df.info()

In [0]:
#!cp steel-defect/train.csv -r steel-defect/train/train.csv

In [0]:
import cv2
def get_steel_dict(img_dir):
  df=pd.read_csv(img_dir+'/train.csv')
  df['EncodedPixels']=df['EncodedPixels'].apply(lambda x:ast.literal_eval(x))
  df['ClassId']=df['ClassId'].apply(lambda x:ast.literal_eval(x))
  df=df.iloc[0:10,:]#remove slicing for get all data in dataframe
  # iterating over rows using iterrows() function  
  dataset_dicts = []

  for index, row in df.iterrows(): 
        record = {}
        filename = os.path.join(img_dir,row['ImageId'])
        height, width = cv2.imread(filename).shape[:2]
        record["file_name"] = filename
        record["image_id"] = index
        record["height"] = height
        record["width"] = width
        #print(record)
        mask = build_mask(encodings=row['EncodedPixels'], labels=row['ClassId'])
        data_list=get_contour_train(file_name=row['ImageId'], mask=mask)#get conour image_id,class_id for val change->get_contour_train to get_contour_val
        
        #show Data
        objs = []
        for data in data_list:
          for array in data['contour'][0]:
            array=array.reshape((array.shape[0],array.shape[-1]))
            #print("Image id=",data['Image_id'][0])
            #print("Class Id=",data['Class_id'][0])
            px=[i[0] for i in array]
            py=[j[1] for j in array]

            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                #"bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": data['Class_id'][0],
                "iscrowd": 0
            }

            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)    
  return dataset_dicts

In [0]:
dataset_dicts=get_steel_dict('steel-defect/train')

In [0]:
dataset_dicts